In [1]:
using Nemo

In [2]:
include("subresultants.jl")

subresultants (generic function with 2 methods)

In [4]:
R, (x,y) = polynomial_ring(QQ, [:x, :y])
f = 1594627541097114368612385*x^16 + 24126594979283487048484560//18765*x^15*y - 648897523515553246784074808*x^14*y^2 + 9227671981881181765203810480*x^13*y^3 + 775388988595641728298874736332*x^12*y^4 + 10787151407096126900712389934720*x^11*y^5 + 77139559828286619230682855993096*x^10*y^6 + 452145503210946168432702707056320*x^9*y^7 + 1603450939087454055697523432885078*x^8*y^8 + 472422761323888641034023872620080*x^7*y^9 + 69254293553970771431091788378664*x^6*y^10 + 8171004998268972015374643672912*x^5*y^11 + 486748426688048642456485687660*x^4*y^12 - 12064621488868591078399181568*x^3*y^13 - 1468208821458887902925840024*x^2*y^14 + 12892282240123223587086528*x*y^15 + 1551343473382248651380209*y^16 - 7416582067989458752697004*x^14 - 86407846455606538783280976*x^13*y + 2644747193988832166794696620*x^12*y^2 + 4962811393512179992436546328*x^11*y^3 - 852556155710875435151137729948*x^10*y^4 - 9006223258524272797865687202672*x^9*y^5 - 37408868951040006789841449492372*x^8*y^6 - 96029136974599596104658658159536*x^7*y^7 - 25805546034923632520724584708244*x^6*y^8 - 5242437191813955076157431442064*x^5*y^9 - 409699928605220782678759186860*x^4*y^10 + 39375118017080555511082417752*x^3*y^11 + 4414478650057190403833967932*x^2*y^12 - 62266637085710610417717552*x*y^13 - 7665646840583710272384908*y^14 + 14233446085691566995553618*x^12 + 132571431253302115292123136*x^11*y - 3677320627079906075667837422*x^10*y^2 - 28335523591694036134043888352*x^9*y^3 + 186478273436442358105601202678*x^8*y^4 + 979680543110585521095162296976*x^7*y^5 - 2082167356121156563744172163572*x^6*y^6 - 30539225710419394554664076544*x^5*y^7 + 25429103844006118137220734342*x^4*y^8 - 40056866529454773916181059488*x^3*y^9 - 4741918132209313788162640030*x^2*y^10 + 120063392649421566997191792*x*y^11 + 15453863115268541628254722*y^12 - 14508062550440954802512688*x^10 - 115292786706841839957190896*x^9*y + 2178406370189758736513057100*x^8*y^2 + 17911006599365461837132806264*x^7*y^3 + 18721721306068791970712906740*x^6*y^4 + 140144222573062968441350663256*x^5*y^5 + 15151268277268342933019382004*x^4*y^6 + 14388997206020218965816744888*x^3*y^7 + 2181175955893013097832336108*x^2*y^8 - 116823607901494850142791760*x*y^9 - 16262571151436698863824960*y^10 + 8412196412628079801200763*x^8 + 60038503306269383516796672*x^7*y - 526872554079154536701276538*x^6*y^2 - 2627205980235442180204133952*x^5*y^3 + 1415955761616170935932891503*x^4*y^4 - 1207248891865676270007025776*x^3*y^5 - 411102077811099614734692666*x^2*y^6 + 59338125380312133254699856*x*y^7 + 9471776668272238878375147*y^8 - 2755131867388737857954960*x^6 - 16865677220581290594690288*x^5*y + 37152628773628751552853580*x^4*y^2 - 32454665768852564904473352*x^3*y^3 + 31405824474395795210222812*x^2*y^4 - 14499431708247069028795056*x*y^5 - 2982642484703489520084064*y^6 + 476197729395824935891282*x^4 + 1850003538678786657657456*x^3*y - 861756868723874605906006*x^2*y^2 + 1328290412641467896291664*x*y^3 + 466653249763538900703730*y^4 - 37550891878150349195068*x^2 - 32144769403583975845728*x*y - 33575411625230211328108*y^2 + 864999024811562109121
fy = derivative(f,y)

1608439665285565803232304//1251*x^15 - 1297795047031106493568149616*x^14*y + 27683015945643545295611431440*x^13*y^2 - 86407846455606538783280976*x^13 + 3101555954382566913195498945328*x^12*y^3 + 5289494387977664333589393240*x^12*y + 53935757035480634503561949673600*x^11*y^4 + 14888434180536539977309638984*x^11*y^2 + 132571431253302115292123136*x^11 + 462837358969719715384097135958576*x^10*y^5 - 3410224622843501740604550919792*x^10*y^3 - 7354641254159812151335674844*x^10*y + 3165018522476623179028918949394240*x^9*y^6 - 45031116292621363989328436013360*x^9*y^4 - 85006570775082108402131665056*x^9*y^2 - 115292786706841839957190896*x^9 + 12827607512699632445580187463080624*x^8*y^7 - 224453213706240040739048696954232*x^8*y^5 + 745913093745769432422404810712*x^8*y^3 + 4356812740379517473026114200*x^8*y + 4251804851914997769306214853580720*x^7*y^8 - 672203958822197172732610607116752*x^7*y^6 + 4898402715552927605475811484880*x^7*y^4 + 53733019798096385511398418792*x^7*y^2 + 60038503306269383516796672*x^7 + 692542935539707714310917883786640*x^6*y^9 - 206444368279389060165796677665952*x^6*y^7 - 12493004136726939382465032981432*x^6*y^5 + 74886885224275167882851626960*x^6*y^3 - 1053745108158309073402553076*x^6*y + 89881054980958692169121080402032*x^5*y^10 - 47181934726325595685416882978576*x^5*y^8 - 213774579972935761882648535808*x^5*y^6 + 700721112865314842206753316280*x^5*y^4 - 7881617940706326540612401856*x^5*y^2 - 16865677220581290594690288*x^5 + 5840981120256583709477828251920*x^4*y^11 - 4096999286052207826787591868600*x^4*y^9 + 203432830752048945097765874736*x^4*y^7 + 90907609663610057598116292024*x^4*y^5 + 5663823046464683743731566012*x^4*y^3 + 74305257547257503105707160*x^4*y - 156840079355291684019189360384*x^3*y^12 + 433126298187886110621906595272*x^3*y^10 - 360511798765092965245629535392*x^3*y^8 + 100722980442141532760717214216*x^3*y^6 - 6036244459328381350035128880*x^3*y^4 - 97363997306557694713420056*x^3*y^2 + 1850003538678786657657456*x^3 - 20554923500424430640961760336*x^2*y^13 + 52973743800686284846007615184*x^2*y^11 - 47419181322093137881626400300*x^2*y^9 + 17449407647144104782658688864*x^2*y^7 - 2466612466866597688408155996*x^2*y^5 + 125623297897583180840891248*x^2*y^3 - 1723513737447749211812012*x^2*y + 193384233601848353806297920*x*y^14 - 809466282114237935430328176*x*y^12 + 1320697319143637236969109712*x*y^10 - 1051412471113453651285125840*x*y^8 + 415366877662184932782898992*x*y^6 - 72497158541235345143975280*x*y^4 + 3984871237924403688874992*x*y^2 - 32144769403583975845728*x + 24821495574115978422083344*y^15 - 107319055768171943813388712*y^13 + 185446357383222499539056664*y^11 - 162625711514366988638249600*y^9 + 75774213346177911027001176*y^7 - 17895854908220937120504384*y^5 + 1866612999054155602814920*y^3 - 67150823250460422656216*y

In [5]:
res = (subresultants(f,fy,2)[1])
#D = collect(factor_squarefree(res))

632559895040048450895748090921641756886470228306004817227440309623067873398228088069018040251313080535541450992745300209771913835521779702851008199977671534167020726336028849097062217287114441992687321230930303095905972747640853160351660208664959050500217080574852996961414727994443272751786036434872954875250716232070420773759333628797461020047048235438883314149281419343763992629189899241992642798646724391513499355847851338387200722573897704432812196619126617722623406622592983236145079741904401558388992742690851903641091398722543025648433824160021835136234674310742569983816952022083521533801084276938125707406682530227862453357465307279164636566301531780944075120368014853781399659352324804488911308378478698006412255396289309228753762567885043144255897027389133906655733401960480165347379506923302457102765690367959194389553475275566215856622522401289230737466554330152659753698941296424927480901117973044723248858530377801520232634035287951690170368//444254602912010649973439519576625850587386574321*x^240 - 27280401809043582954539709800229226941922256980762099535036499469356024225192847844918313932768028629251329437519101996993970341851293586870895549640347066844386299636910638649798335456186372490446008392012158871353916959299022590903163552882931093272444691187445341146648517946361167018968136826774772113962849465879790102455251348737056479322221709995316094078136416250360990523364461729461850028858705264806880923150961924124407046423932721077201463196081164343322738702781866248034507095968507453401568130160667467602453149314651008726972613943280037431840411263647639241806686969966910099416809803044475402362256227285290811958827145608766272617475011223862153240270515220786858552113698761257356726448825101429564119037578085319198826291782685401319262165064153604466681776942253520673005988389961671405814531881624435876753933251290637808429188046672563460115471178341729235324494283405530920355753033727568907783250422603641548562131288533980151808//1065358759980840887226473668049462471432581713*x^238 + 232781727688990465067222477489171853922170824222830873466758849005239573328709048867518094392804620531802515089304440789999706741203247359411010880719507536004782263325736137280512250590150966116095228081518608802857473947793747485046412665305104447231081157234793691795701848519131673943242523376269499607036769689817868950339864725301942561716706029279975016955889239618870902178157502712755493624125552228706266863442546954850305100467544450803962403261330733415929264661647189816796642033870045716452603276388132054172372105288655556215235864970809421544949965692582195225570574030007138579183752943602616698304283826540504098824790923002425697513101027975260447409809278811131493861078180430317826111038091879066104658578924134511286571046125616886493327306941169067384123338683452219957273712037287725185724815721416507176740389822525808772782494712089600290416123142198812054291146550417774558245587602317169015805421208868749622046871915591312080896//1065358759980840887226473668049462471432581713*x^236 - 375899300053045836676578395754950676074513638605130734822374807133807961151912783306572747423213196160490522973008839832109627864381070303154691182243730279729087051582756823850226943873567163786054954705629196354903150290364886051206057164564839184724928245086562134756977285380800238506590497511497410695801781062281546620286174254177052059203902044998233254400037895760537642423211599328490596696093138849435922327235969959958655727124461455094385687553692966657976615683093462198774913137257662846147004953661599801833055982557781824175547928121693348085783288965750404755953256886674757010965748308292466256876192824320313804330677586198144517107788377531201398752993687113610763775936666249499816993477655052628990237412865380561708390433252555917653989082483036395083837358806114650192142092463712073863424604821968571445076375235310779303698117451695712044860031378628844292510112084894962498888067289554659901832708135329383201297751536361717471641

In [9]:
function subresultants(P::PolyRingElem{T}, Q::PolyRingElem{T}) where T <: RingElement
    if degree(P) < degree(Q)
        P, Q = Q, P
    end
    S = []
    s = leading_coefficient(Q)^(degree(P)-degree(Q))
    A = Q
    B = pseudorem(P,-Q)
    ring = parent(P)
    while true
        d = degree(A)
        e = degree(B)
        if iszero(B)
            return S
        end
        pushfirst!(S, copy(B))
        delta = d - e
        if delta > 1
            if length(S) > 1
                n = degree(S[2]) - degree(S[1]) - 1
                if n == 0
                    C = copy(S[1])
                else
                    x = leading_coefficient(S[1])
                    y = leading_coefficient(S[2])
                    a = 1 << (length(bits(ZZ(n))) - 1)
                    c = x
                    n = n - a
                    while a > 1
                        a ÷= 2
                        c = c^2 / y
                        if n >= a
                            c = c * x / y
                            n = n - a
                        end
                    end
                    C = c * S[1] / y
                end
            else
                C = leading_coefficient(B)^(delta-1) * B / s^(delta-1)
            end
            pushfirst!(S, copy(C))
        else
            C = copy(B)
        end
        if e == 0
            return S
        end
        B = pseudorem(A,-B) / (s^delta * leading_coefficient(A))
        A = copy(C)
        s = leading_coefficient(A)
    end
end


subresultants (generic function with 1 method)

In [7]:
#using Nemo
R, x = polynomial_ring(QQ, "x")
#f = x^8 + x^6 -3*x^4 -3*x^3 +8*x^2 +2*x -5
#g = 3*x^6 +5*x^4 -4*x^2 -9*x +21
S, y = polynomial_ring(R, "y")
f = (y^2 + 6)*(x - 1) - y*(x^2 + 1)
g = (x^2 + 6)*(y - 1) - x*(y^2 + 1)

-x*y^2 + (x^2 + 6)*y - x^2 - x - 6

In [6]:
subresultants(f,g)

UndefVarError: UndefVarError: `g` not defined

In [13]:
divrem(f,-g)[2]

ArgumentError: ArgumentError: not an exact division

In [14]:
prevpow2(ZZ(100))

64

In [15]:
parent(degree(f))

Integers

In [16]:
RR, (a,b) = Nemo.polynomial_ring(QQ,[:a,:b])

(Multivariate polynomial ring in 2 variables over QQ, QQMPolyRingElem[a, b])

a^2 + a*b^2 + a*b + b

In [196]:
function poly_to_array(P::MPolyRingElem)
    # return P as two lists ci and vi of resp. coeffs and exponents
    return [collect(coefficients(P)), collect(exponent_vectors(P))]
end

function rem_ind(L,i)
    # remove element at index i
    return [ L[1:i-1]; L[i+1:end] ]
end

function add_ind(L,i,x)
    # add element x at index i
    return [ L[1:i-1]; [x]; L[i:end] ]
end

function parray_asvar(LP, idx)
    # takes a sparse representation of a poly P  
    # and outputs a sparse rpz of P seen as a univariate poly in the i-th variable
    deg = sort(LP[2], by= x->x[idx])[end][idx]
    NLP = [ [[],[]] for _ in 1:deg+1 ]
    for i in eachindex(LP[2])
        di = LP[2][i][idx]+1
        push!(NLP[di][1], LP[1][i])
        push!(NLP[di][2], rem_ind(LP[2][i], idx)) 
    end
    return NLP
end

function array_to_poly(LP, A)
    C = MPolyBuildCtx(A)
    R = base_ring(A)
    for i in eachindex(LP[1])
        push_term!(C, R(LP[1][i]), LP[2][i]);
    end
    return finish(C)
end

function asvar_to_parray(NLP, idx)
    LP = [[],[]]
    for i in 1:length(NLP)
        for j in eachindex(NLP[i][1])
            push!(LP[1], NLP[i][1][j])
            push!(LP[2], add_ind(NLP[i][2][j], idx, i-1))
        end
    end
    return LP
end

asvar_to_parray (generic function with 1 method)

In [133]:
Q = a*b+1a

a*b + 1

In [217]:
function subresultants(P::MPolyRingElem{T}, Q::MPolyRingElem{T}, idx) where T <: RingElement
    LPQ = map(poly_to_array, [P,Q])
    ULPQ = [ parray_asvar(lpq, idx) for lpq in LPQ ]

    R, x = polynomial_ring(base_ring(parent(P)), "x")
    S, y = polynomial_ring(R, "y")

    UP, UQ = [ S([ array_to_poly(l, R) for l in lpq ]) for lpq in ULPQ ]

    sr = subresultants(UP, UQ)

    # Get it back to initial polynomial ring
    Lsr = [ [collect(coefficients(csr)) for csr in coefficients(sr)] for sr in sr]
    newsr = []
    for lsr in Lsr
        mlsr = [[],[]]
        for i in 1:length(lsr)
            for j in 1:length(lsr[i])
                push!(mlsr[1], lsr[i][j])
                push!(mlsr[2],add_ind([j-1],idx, i-1))
            end
        end
        push!(newsr, array_to_poly(mlsr, parent(P)))
    end

    return newsr
end

subresultants (generic function with 3 methods)

In [2]:
include("subresultants.jl")

subresultants (generic function with 2 methods)

In [25]:
R, (x,y) = polynomial_ring(ZZ, [:x,:y])
f = (y^2 + 6)*(x - 1) - y*(x^2 + 1)
g = (x^2 + 6)*(y - 1) - x*(y^2 + 1)
idx = 2
subresultants(x,x,1)

Multivariate polynomial ring in 2 variables x, y
  over integer ring

In [9]:
res = subresultants(f,g,idx)

2-element Vector{Any}:
 2*x^6 - 22*x^5 + 102*x^4 - 274*x^3 + 488*x^2 - 552*x + 288
 -x^3 - x^2*y + 6*x^2 + 5*x*y - 11*x - 6*y + 6

In [24]:
sq = collect(factor_squarefree(res[1]))


MethodError: MethodError: no method matching keys(::Fac{ZZMPolyRingElem})

Closest candidates are:
  keys(!Matched::Pkg.Registry.RegistryInstance)
   @ Pkg /opt/julia-1.10.1/share/julia/stdlib/v1.10/Pkg/src/Registry/registry_instance.jl:447
  keys(!Matched::Cmd)
   @ Base process.jl:678
  keys(!Matched::Core.SimpleVector)
   @ Base essentials.jl:781
  ...


In [204]:
NLP = parray_asvar(LP,1)
asvar_to_parray(NLP,1)


2-element Vector{Vector{Any}}:
 [1, 1, 1, 1]
 [[0, 1], [1, 2], [1, 1], [2, 0]]

In [128]:
S([ array_to_poly(lp, R) for lp in NLP ])

y^2 + (x^2 + x)*y + x

a^2 + a*b^2 + a*b + b

In [106]:
idx=1
sort(LP[2], by= x->x[idx])

4-element Vector{Vector{Int64}}:
 [0, 1]
 [1, 2]
 [1, 1]
 [2, 0]

Base.Generator{UnitRange{Int64}, typeof(identity)}(identity, 1:2)